In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def buscar_produto_kabum(navegador, produto):
    # Abre a página da Kabum
    navegador.get("https://www.kabum.com.br")

    try:
        # Busca o campo de pesquisa e realiza a busca
        campo_busca = WebDriverWait(navegador, 15).until(
            EC.presence_of_element_located((By.ID, "input-busca"))
        )
        campo_busca.send_keys(produto)
        campo_busca.submit()

        # Espera os resultados carregarem e pega o número de produtos
        num_resultados = WebDriverWait(navegador, 15).until(
            EC.presence_of_element_located((By.ID, "listingCount"))
        ).find_element(By.TAG_NAME, "b").text
        
        total_produtos_kabum = int(num_resultados)

        # Coletando os preços
        precos_kabum = navegador.find_elements(By.CLASS_NAME, "priceCard")
        total_preco_kabum = 0.0
        
        for preco in precos_kabum:
            valor = preco.text.replace('R$', '').replace('.', '').replace(',', '.').strip()
            total_preco_kabum += float(valor)

        print(f"Busca realizada com sucesso na Kabum para o produto: {produto}. Total de resultados: {total_produtos_kabum}. Valor total: R$ {total_preco_kabum:.2f}")
    except Exception as e:
        print("Erro ao buscar o produto na Kabum:", e)

def buscar_produto_magalu(navegador, produto):
    # Abre uma nova aba e vai para a Magalu
    navegador.execute_script("window.open('https://www.magazineluiza.com.br', '_blank');")
    navegador.switch_to.window(navegador.window_handles[1])  # Muda para a nova aba

    try:
        # Busca o campo de pesquisa e realiza a busca
        campo_busca = WebDriverWait(navegador, 15).until(
            EC.presence_of_element_located((By.ID, "input-search"))
        )
        campo_busca.send_keys(produto)

        # Clica no ícone de busca
        botao_busca = WebDriverWait(navegador, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='search-submit']"))
        )
        botao_busca.click()

        # Espera os resultados carregarem e pega o número de produtos
        num_resultados = WebDriverWait(navegador, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "sc-hjpnKN"))
        ).text
        
        total_produtos_magalu = int(''.join(filter(str.isdigit, num_resultados)))

        # Coletando os preços, limitando a 100 itens
        precos_magalu = navegador.find_elements(By.CSS_SELECTOR, "[data-testid='price-value']")
        total_preco_magalu = 0.0
        
        for index, preco in enumerate(precos_magalu):
            if index >= 100:  # Limita a 100 itens
                break
            
            # Limpa o texto e extrai apenas o valor numérico
            valor = preco.text.replace('R$', '').replace('.', '').replace(',', '.').strip()
            valor = valor.split('ou')[-1].strip()  # Extrai a parte após "ou" se existir

            if valor:  # Verifica se o valor não está vazio
                total_preco_magalu += float(valor)

        print(f"Busca realizada com sucesso na Magalu para o produto: {produto}. Total de resultados: {total_produtos_magalu}. Valor total: R$ {total_preco_magalu:.2f}")
    except Exception as e:
        print("Erro ao buscar o produto na Magalu:", e)

# Função principal para executar as buscas
def buscar_produtos(produto):
    # Inicia o navegador
    navegador = webdriver.Chrome()
    
    # Busca na Kabum
    buscar_produto_kabum(navegador, produto)
    
    # Busca na Magalu
    buscar_produto_magalu(navegador, produto)

    # Mantém o navegador aberto
    print("O navegador está aberto com ambas as guias.")
    input("Pressione Enter para fechar o navegador.")
    navegador.quit()  # Fecha o navegador quando terminar

# Chamada da função com o produto desejado
buscar_produtos("cadeira")